#  Convolutional Neural Network Tutorial

Mit diesem Notebook bietet eine erste einfache Einführung in die Verwendung von Convolutional Neural Networks.
Es werden alle wichtigen Schritte von durchgeführt und beschrieben die notwendig sind um ein Model zu erstellen, Trainingsdaten zu laden, das Model zu trainieren und anschließend zu bewerten.

Es wird ein CNN als binärer Klassifikator trainiert werden um Chihuahuas von Muffins zu unterscheiden. 
<img src="Notebook_Images/full.jpg">

https://medium.com/ibm-data-science-experience/markdown-for-jupyter-notebooks-cheatsheet-386c05aeebed

## Vorbereitungen

    1. Python installieren: https://www.python.org/downloads/
    2. Jupyter installiern: http://jupyter.org/install
        In commandprompt:
        python3 -m pip install --upgrade pip
        python3 -m pip install jupyter

## Laden der Bilder
Zunächst müssen die Daten zum Training geladen werden. Die einzelnen Dateien müssen dafür nach Klassen sortiert in Ordner vorhanden sein. <br>
* \Data
   * \Chihuahuas 
      *  img1.jpg
      *  img2.jpg
      *  img3.jpg 
   * \Muffins
       * img1.jpg 
       * img2.jpg 
       * img3.jpg 

Bilddateien aus diesen Ordnern werden geladen, skaliert und ein Array mit entsprechenden Labels erstellt. 

In [ ]:
import numpy as np

image_height=172
image_width= 171

In [ ]:
#Diese Datei ist Teil des Repositorys
from dataset import DataLoader
#Pfad zu den Ordnern mit den Bildern 
data_dir = '.\\Images\\chihuahua-muffin\\train'

testset= DataLoader()
images, labels = testset.load_data(data_dir, image_height,image_width)
images, labels = testset.normalizeData(images, labels)

### Datenset speichern in einer npz-Datei
Um die Bilder nicht jedesmal neu laden zu müssen, stellt numpy eine sehr effizente Methode zur Verfügung um große Datenmengen als .npz-Datei zu speichern. Dafür müssen der Methode die Bilder und Labels als Numpy-Array übergeben werden. 

In [ ]:
filename= 'dog-vs-muffins.npz'
np.savez_compressed(filename,
                    images=images,
                    labels=labels)

### Laden der Daten aus einer npz-Datei
Die Bilder und Labels können nach öffnen der Datei geladen werden. 

In [ ]:
dataset= np.load('dog-vs-muffins.npz')
labels=dataset['labels']
images=dataset['images']
images= np.concatenate((images[labels ==0][:100], images[labels ==1][:100]) ,axis=0)
labels= np.concatenate((labels[labels ==0][:100], labels[labels ==1][:100]) ,axis=0)
print('load %d files' %len(labels))

### Anzeigen einiger Testbilder
Die Plot-Methode wird in diesem Notebook verwendet um die Bilder anzuzeigen. In dem ersten Beispiel werden je acht Bilder beider Klassen angezeigt werden

In [ ]:
import matplotlib.pyplot as plt
images_to_show= np.concatenate((images[labels ==0][:8], images[labels ==1][:8]) ,axis=0)
print(len(images_to_show))

def plot(images_to_plot, predictions= np.array([]), titles=[]):
    fig = plt.figure(figsize=(10, 10))
    for i in range(len(images_to_plot)):
        ax = fig.add_subplot(4, 4, 1 + i, xticks=[], yticks=[])
        im = images_to_plot[i]
        if titles:
            ax.set_title(titles[i])
        if predictions.size:
            ax.set_xlabel('dog: %6.2f \n muffin %6.2f'
                      % (predictions[i][0], predictions[i][1]), fontsize=12)
        plt.imshow(im)
    
    plt.tight_layout()
    plt.show()

plot(images_to_show)

## Vorbereitung der Daten

### Image Augmentation
Zur Vergrößerung der Datenmenge werden die Images bearbeitet. Für dieses Beispiel werden die Bilder vertikal oder horizontal gespiegelt zusätzlich wird von dem ursprünglichen Bild noch eine Kopie als Grauwertbild erzeugt.

In [ ]:
import cv2

img= images[0]

horizontal_flipt= cv2.flip(img, 0)
vertical_flipt= cv2.flip(img, 1)
gray_scaled_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

augmented_images= [img, horizontal_flipt, vertical_flipt, gray_scaled_img]
titels=['original', 'horizonzal', 'vertical', 'grayscaled']
plot(augmented_images, titles=titels)

def randomize_images(images,labels):

    for index in range(len(images)):
        img = images[index]
        flipt_img= cv2.flip(img, np.random.randint(0,1))
        flipt_img = np.expand_dims(flipt_img, axis=0)
        images=np.vstack((images, flipt_img))
        labels = np.append(labels, np.asarray(labels[index]))
    print(len(images))
    return images, labels

Nach dem vorbereiten der Daten werden die Arrays nach dem Zufallsprinzip durchmischt. Die sklearn.shuffel Methode durchmischt die Bilder und Labels gleichermaßen.Zur Reproduzierbarkeit wird der Random_State auf einen konstanten Wert gesetzt.

In [ ]:
from sklearn.utils import shuffle
images , labels = shuffle(images , labels, random_state=9)

Anschließend werden die Daten in eine Trainings-, Validierungs- und Teststichprobe aufgeteilt. 
Trainings und Validierungsstichprobe werden zum anpassen der Gewichte beim Lernen verwendet. 
Die Teststichprobe muss vor der Trainingsphase abgespalten werden und dient dazu den Klassifikator nach dem Training 
zu bewerten.
<img src="Notebook_Images/Test-train-Val.PNG" height="600" width="600" >
<cite data-cite="Raschka">(Sebastian Raschka und Vahid Mirjalili. Machine Learning mit Python
und Scikit-learn und TensorFlow: Das umfassende Praxis-Handbuch für
Data Science, Deep Learning und Predictive Analytics. 2., aktualisierte
und erweiterte Auflage. Frechen: mitp, 2018. isbn: 978-3-95845-733-1.)</cite>


In [ ]:
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

length=len(labels)

X_train = images[:int(length*0.8)]
X_val   = images[int(length*0.8):int(length*0.9)]
X_test  = images[int(length*0.9):]
y_train = labels[:int(length*0.8)] 
y_val   = labels[int(length*0.8):int(length*0.9)]
y_test  = labels[int(length*0.9):]


y_train= np_utils.to_categorical(y_train, 2)
y_val = np_utils.to_categorical(y_val, 2)

print("Trainingsdaten:", X_train.shape)
print("Validierungsdaten:", X_val.shape)
print("Testdaten:", X_test.shape)

### Standardisierung der Pixelwerte
Um zu verhindern, dass Ausreißer in den Pixelwerten einen überproportional großen Einfluss bei der Gewichtsänderung haben, werden die Merkmalswerte standartisiert. 

\begin{equation*}
x^{(i)}_{std}=\frac{x^{(i)}-\mu_x}{\sigma_x}
\end{equation*}

<cite data-cite="Raschka">(Sebastian Raschka und Vahid Mirjalili. Machine Learning mit Python
und Scikit-learn und TensorFlow: Das umfassende Praxis-Handbuch für
Data Science, Deep Learning und Predictive Analytics. 2., aktualisierte
und erweiterte Auflage. Frechen: mitp, 2018. isbn: 978-3-95845-733-1.)</cite>

In [ ]:
mean_values=np.mean(X_train, axis=0)
std_values=np.std(X_train)

x_train_centered= (X_train- mean_values)/std_values
x_val_centered= (X_val- mean_values)/std_values
x_test_centered= (X_test- mean_values)/std_values

# Convolutional Neural Network

Mit diesem Befehl wird ein Convolutional-Layer in dem Model erzeugt. Dazu müssen die Anzahl der Filter =32 und die Größe der Filterkerne in Pixel=5x5 sowie die Aktivierungsfunktion activation='relu' angegeben werden. Optional kann die Initalisierung und der Padding Modus angegeben werden.  
    * model.add(Convolution2D(32, (5, 5), activation='relu',kernel_initializer='glorot_uniform', padding='same'))

Hiermit wird nach einem Convolutional-Layer ein Pooling-Layer der Größe 2x2 Pixel angehangen.
    * model.add(MaxPooling2D(pool_size=(2, 2)))

Dense-Layer sind Vollverknüpfte Schichten. Hier im Beispiel mit 512 Neuronen auf dieser Schicht. Durch die Zahl der Neuronen des letzten Dense-Layer wird die Zahl der zu unterscheidendenden Klassen bestimmt.
    * model.add(Dense(512,kernel_initializer='glorot_uniform', activation='relu'))
    
Zuletzt muss das Model kompiliert werden. In diesem Fall wird als Loss-Funktion Categorical-Crossentropy verwendet, für ein Zweiklassen-Problem kann Binaray-Crossentropy verwendet werden. Als Optimierungsverfahren ist Stochhastical Gradient Descent angegeben, die Parameter wie Lernrate (lr) können während des Lernens variiert werden. 
    * model.compile(optimizer=SGD(lr=0.001, momentum=0.9), loss='categorical_crossentropy',  metrics=['accuracy'])
    
Der Befehl model.summary gibt die Struktur des Modells in der Console aus.

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D,  BatchNormalization, Dropout, Flatten, Dense
from keras.optimizers import SGD

model = Sequential()

model.add(Convolution2D(32, (5, 5), activation='relu',kernel_initializer='glorot_uniform', padding='same', 
                        input_shape=(image_height, image_width, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(32, (3, 3),kernel_initializer='glorot_uniform', activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(32, (3, 3),kernel_initializer='glorot_uniform', activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512,kernel_initializer='glorot_uniform', activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(128,kernel_initializer='glorot_uniform', activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer=SGD(lr=0.001, momentum=0.9), loss='categorical_crossentropy',  metrics=['accuracy'])
model.summary()

### Training

Der Fit-Befehl trainiert das Modell. Dazu müssen Training- und Validierungsdaten sowie die Anzahl der Trainingsepochen übergeben werden. Mit dem Verbose Parameter kann bestimmt werden wie oft während des Trainings eine Konsolenausgabe mit aktuellen Trainingswerten stattfinden soll. Callbacks nimmt eine Liste von zuvor definierten Callback-Methoden entgegen, in unserem Fall wurde ein Callback definiert welcher während des Trainings die aktuellen Gewichtungen speichert sobalb die Genauigkeit in einer Epoche gestiegen und der Loss gefallen ist. Es können aber auch Callbacks definiert werden um zum Beispiel bestimmte Werte zur späteren Visualisierung zu speichern oder die Lernrate adaptiv anzupassen.

In [ ]:
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='model.hdf5', verbose=1, save_best_only=True)


model.fit(x_train_centered, y_train,
          batch_size=100,
          epochs=1,
          validation_data=(x_val_centered,  y_val),
          verbose=1,
          callbacks=[checkpointer])

### Laden eines bereits trainierten Modells
Modelle können während und nach dem Lernvorgang zur späteren Wiederverwendung gespeichert werden. Keras bietet hierfür mehrere Formate wie JSON oder hdf5 an. Diese Modelle können anschließend wieder geladen werden um neue Daten zu klassifizieren.

In [ ]:
from keras.models import load_model
model=load_model("myModel.hdf5")

### Evaluation

Die Predict Methode wird verwendet um mit dem Modell die Klassenwahrscheinlichkeiten von Bildern zu bestimmen, dazu können der Methode einzelne Bilder oder ein Array von Bildern übergeben werden. 

Die Evaluate Methode dient als erste schnelle Methode um ein Modell zu bewerten, den diese Methode bestimmt Genauigkeit und Loss für ein Array an Daten. 

Zu Beginn wurden die Bilder in Trainings- Validierungs- und Teststichprobe aufgeteilt. Die Teststichprobe kam beim Training nicht zum Einsatz und ist somit für das Modell noch unbekannt. Dadurch kann sie zur Evaluation des Modells dienen.

Anschließend werden die Klassenwahrscheinlichkeiten für die eigentlichen Produktivdaten ermittelt. Dazu werden diese zunächst geladen und standardisiert.

In [ ]:
img_to_plot= X_test[:16]
predictions= model.predict(x_test_centered)[:16]
plot(img_to_plot, predictions= predictions)

true_labels=np_utils.to_categorical(y_test, 2)
metric= model.evaluate(x_test_centered, true_labels)

print('loss: %10.2f  \naccuracy: %6.2f' % (metric[0], metric[1]))

In [ ]:
dataset= np.load('test_data.npz')
labels=dataset['labels']
images=dataset['images']

mean_values=np.mean(X_train, axis=0)
std_values=np.std(X_train)

x_test_centered= (images- mean_values)/std_values
plot(images, predictions= model.predict(x_test_centered))

true_labels=np_utils.to_categorical(labels, 2)
metric= model.evaluate(x_test_centered, true_labels)

print('loss: %10.2f  \naccuracy: %6.2f' % (metric[0], metric[1]))